## 1. Data Exploration

In [ ]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import seaborn as sns
import os
from PIL import Image

from keras.utils import image_dataset_from_directory

from sklearn.model_selection import train_test_split
import shutil


from tensorflow.keras import Model
from tensorflow.keras.layers import Rescaling, Dense
from tensorflow.keras.applications import MobileNetV2


import tensorflow as tf
from keras.layers import Input, Rescaling, Dense

import random


from sklearn.model_selection import StratifiedKFold
from cleanlab.models.keras import KerasWrapperModel


In [ ]:
os.getcwd()

In [ ]:
# Set the desired path
path = "/Users/joaohenriques/Desktop/Mestrados/Data Science Nova IMS/My docs/2nd sem/Deep Learning/Project/DL_project_Group33"

# Change the working directory
os.chdir(path)

# Optional: Confirm
print("Current directory:", os.getcwd())

In [ ]:
# Load metadata
metadata_path = 'data/metadata.csv'
metadata = pd.read_csv(metadata_path)

# Verify data loaded correctly
metadata.head()

## 1.1 Data Integrity Checks

Check for number of images

In [ ]:
num_images = len(metadata)
print(f"Number of images: {num_images}")

Check if all paths in metadata are valid

In [ ]:
# Check if each file listed in metadata actually exists
metadata['full_path'] = 'data/' + metadata['file_path']
metadata['file_exists'] = metadata['full_path'].apply(os.path.exists)

# Summary of missing files
missing_files = metadata[~metadata['file_exists']]
print(f"Missing files count: {len(missing_files)}")
if not missing_files.empty:
    display(missing_files.head())

Check for corrupted images (Can we open all of them)

In [ ]:
# source: https://drlee.io/identifying-corrupted-images-before-feeding-them-into-a-cnn-13397844ef3c
def is_image_valid(path):
    try:
        Image.open(path).verify()
        return True
    except:
        return False

metadata['is_valid_image'] = metadata['full_path'].apply(is_image_valid)

corrupted = metadata[~metadata['is_valid_image']]
print(f"Number of corrupted images: {len(corrupted)}")

Check for duplicate paths

In [ ]:
# source: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html
duplicates = metadata[metadata.duplicated(['full_path'], keep=False)]
print(f"Number of duplicate images: {len(duplicates)}")
if not duplicates.empty:
    print("Duplicate images:")
    display(duplicates[['full_path', 'file_path']].head())

Check for empty images (zero-byte files)

In [ ]:
# check for empty images - images with size (0, 0)
def is_image_empty(path):
    try:
        img = Image.open(path)
        return img.size == (0, 0)
    except:
        return True
metadata['is_empty_image'] = metadata['full_path'].apply(is_image_empty)
empty_images = metadata[metadata['is_empty_image']]
print(f"Number of empty images: {len(empty_images)}")
if not empty_images.empty:
    print("Empty images:")
    display(empty_images[['full_path', 'file_path']].head())

Check format consistency (do we only have jpg files?):

In [ ]:
# Check for format consistency
def get_image_format(path):
    try:
        img = Image.open(path)
        return img.format # Return the format of the image
    except:
        return None
metadata['image_format'] = metadata['full_path'].apply(get_image_format)

# Counts how many images are in each format.
format_counts = metadata['image_format'].value_counts() 

# Display the counts of each format
print("Image format counts:")
print(format_counts)

In [ ]:
# Check for images with unusual aspect ratios
def is_aspect_ratio_unusual(path):
    try:
        img = Image.open(path)
        width, height = img.size
        aspect_ratio = width / height
        return aspect_ratio < 0.1 or aspect_ratio > 10 # aspect_ratio > 10: very wide and short, aspect_ratio < 0.1: very tall and narrow
    except:
        return True
    
num_unusual = metadata['full_path'].apply(is_aspect_ratio_unusual).sum()
print(f"Number of images with unusual aspect ratios: {num_unusual}")

In [ ]:
# Get distinct rare species ids
species_ids = metadata['rare_species_id'].unique()
print(f"Number of unique species IDs: {len(species_ids)}")


# Display a sample of the images
sample_species_ids = species_ids[:5]


In [ ]:
# Understand the structure of the image files
# 1. Each folder name corresponds to phylum_family name

## 1.2 Data Understanding

### General understanding

General structure

Kindgom --> phylum --> family

In [ ]:
# Number of unique kingdoms, phyla, and families
kingdom_unique = metadata['kingdom'].unique()
phylum_unique = metadata['phylum'].unique()
family_unique = metadata['family'].unique()
print(f"Number of unique kingdoms: {len(kingdom_unique)}")
print(f"Number of unique phylum: {len(phylum_unique)}")
print(f"Number of unique families: {len(family_unique)}")

In [ ]:
# Bar chart: Number of images per phylum 
plt.figure(figsize=(10, 6))
metadata['phylum'].value_counts().plot(kind='bar')
plt.title('Distribution of Phyla')
plt.xlabel('Phylum')
plt.ylabel('Image count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Number of images per kingdom, phylum, and family

# Total number of images
total_images = len(metadata)

# Function to build a summary DataFrame
def build_summary_df(series, label):
    df = series.reset_index()
    df.columns = [label, 'Image Count']
    df['% of Total Images'] = (df['Image Count'] / total_images * 100).round(2)
    return df

# Generate base DataFrames
kingdom_df = build_summary_df(metadata['kingdom'].value_counts(), 'Kingdom')
phylum_df = build_summary_df(metadata['phylum'].value_counts(), 'Phylum')
family_df = build_summary_df(metadata['family'].value_counts(), 'Family')

# Add "% within Phylum" to family_df
# Step 1: Create lookup for phylum per family
family_phylum_map = metadata[['family', 'phylum']].drop_duplicates().set_index('family')['phylum']

# Step 2: Add phylum column to family_df
family_df['Phylum'] = family_df['Family'].map(family_phylum_map)

# Step 3: Get total images per phylum
phylum_image_counts = metadata['phylum'].value_counts()

# Step 4: Calculate % within phylum
family_df['% within Phylum'] = (
    family_df['Image Count'] / family_df['Phylum'].map(phylum_image_counts) * 100
).round(2)

# Optional: Reorder columns
family_df = family_df[['Phylum', 'Family', 'Image Count', '% of Total Images', '% within Phylum']]

display(kingdom_df)
display(phylum_df)
display(family_df)


### Class imbalance detection for phylums

In [ ]:
# Number of families per phylum

# Get number of unique families per phylum
phylum_family_counts = metadata.groupby('phylum')['family'].nunique().reset_index()
phylum_family_counts.columns = ['Phylum', 'Number of Families']

# Calculate total number of families in the entire dataset
total_families = metadata['family'].nunique()

# Add % of total families per phylum
phylum_family_counts['% of Total Families'] = (
    phylum_family_counts['Number of Families'] / total_families * 100
).round(2)

# Add number of images per phylum
phylum_image_counts = metadata['phylum'].value_counts()
phylum_family_counts['Image Count'] = phylum_family_counts['Phylum'].map(phylum_image_counts)

# Sort by image count or number of families
phylum_family_counts = phylum_family_counts.sort_values(by='Number of Families', ascending=False)

# Display
phylum_family_counts


> As we can can see from the results above we have a high imbalance of number of images across the different phylums.

> Echinodermata phylum only has 1 family while the chordata phylum has 166 families

Class imbalance detection for families

In [ ]:
# Understanding the data

family_df.set_index('Family', inplace=True)
family_df

In [ ]:
family_df.describe()

In [ ]:
family_df['Image Count'].hist(bins=20)
plt.xlabel('Image Count')
plt.ylabel('Frequency')
plt.title('Distribution of Image Counts per Family')
plt.show()

In [ ]:
sns.boxplot(family_df['Image Count'])
plt.xlabel('Image Count')
plt.title('Boxplot of Image Counts per Family')
plt.show()

1. Underepresented families
- The minimim value count for images is just one image less than the 50% 
- There doesn't seem to be any underepresented families since the Image count distribution for families seems to be right skewed

2. Overepresented families

- due to the right skewed distribution we will classify overepresented families as families which are above the right bound for outliers (Q3 + 1.5IQR)

Identifying overepresented families

In [ ]:
q3 = family_df['Image Count'].quantile(0.75)
q1 = family_df['Image Count'].quantile(0.25)
iqr = q3 - q1
upper_bound = q3 + 1.5 * iqr
upper_bound

> Considering all the families and differennt phylums, an overepresented family will be considered a family with over 105 images

In [ ]:
overepresented_families = family_df.loc[family_df['Image Count'] > upper_bound]
overepresented_families

> We have a total of 23 overepresented families

Understanding to which phylum the overepresented families family belong to

In [ ]:
overepresented_families['Phylum'].value_counts()

> We can see that most represented families belong to the chordata pylum with a few belonging to the arthropoda and cnidaria

Class imbalance detection for families grouped by Phylum

- By doing this we will be able to locate families that inside their phylum are considered outliers but are not when considering the entire scope
- Able to understand the variability inside each phylum

In [ ]:
# Prepare the data: image count per family + their phylum
family_image_counts = metadata.groupby(['phylum', 'family']).size().reset_index(name='Image Count')

# Plot: Boxplot of image count per family, grouped by phylum
plt.figure(figsize=(14, 6))
sns.boxplot(data=family_image_counts, x='phylum', y='Image Count')
plt.xticks(rotation=45, ha='right')
plt.title('Distribution of Image Counts per Family, Grouped by Phylum')
plt.xlabel('Phylum')
plt.ylabel('Image Count per Family')
plt.tight_layout()
plt.show()


In [ ]:
# Prepare the data: image count per family + their phylum
family_image_counts = metadata.groupby(['phylum', 'family']).size().reset_index(name='Image Count')

# Plot: Boxplot of image count per family, grouped by phylum with mean point
plt.figure(figsize=(14, 6))
sns.boxplot(
    data=family_image_counts,
    x='phylum',
    y='Image Count',
    showmeans=True,                     
    meanprops={"marker": "<", "markerfacecolor": "orange", "markeredgecolor": "black"}
)
plt.xticks(rotation=45, ha='right')
plt.title('Distribution of Image Counts per Family, Grouped by Phylum')
plt.xlabel('Phylum')
plt.ylabel('Image Count per Family')
plt.tight_layout()
plt.show()


> **Conclusions regarding the variability inside each phylum**: As expected chordata (biggest phylum) has the greatest variability in data, having the most extreme outliers (families with very high number of images). Arthopoda and cnidaria are the two next biggest phylums. Even though Arthopoda has more families and more overall images it has less overall variability but a few outliers that might fall below the threshold of 105 (general outliers threshold) that will be inestigated later on. Cnidaria and mollusca offer some variability with only cnidaria having outliers.

Analyzing overepresented families specific for phylum arthropoda

In [ ]:
arthropoda_df = family_df[family_df['Phylum'] == 'arthropoda']
q3 = arthropoda_df['Image Count'].quantile(0.75)
q1 = arthropoda_df['Image Count'].quantile(0.25)
iqr = q3 - q1
upper_bound_arthropoda = q3 + 1.5 * iqr
upper_bound_arthropoda

In [ ]:
# Outliers specifc for arthropoda. Means we are looking for a family with more than upper_bound_arthropoda images but less then the upper_bound
family_df.loc[(family_df['Image Count'] > upper_bound_arthropoda) & (family_df['Image Count'] < upper_bound) & (family_df['Phylum'] == "arthropoda")].sort_values('Image Count', ascending=False).head(10)

> There are 2 families considered overepresented in the arthropoda phylum regarding the image count.
1. family: carabidae 
2. family: cerambycidae


### Image sizes

Identify common image sizes and outliers.
- Detect images with extreme aspect ratios or inconsistent shapes.

In [ ]:
# Detect image size in metadata
metadata['image_size'] = metadata['full_path'].apply(lambda x: Image.open(x).size if os.path.exists(x) else None)

metadata.dtypes 

In [ ]:
metadata.head()

> As you can see the image size is a tuple. We can split it into two columns: width and height. Doing it below

In [ ]:
# Convert image_size tuples into two separate numeric columns
metadata[['image_width', 'image_height']] = pd.DataFrame(metadata['image_size'].tolist(), index=metadata.index)

# Checking datatypes
print(metadata.dtypes[['image_width', 'image_height']])


In [ ]:
# Check for average image width
metadata['image_width'].describe()

In [ ]:
# Check for average image height
metadata['image_height'].describe()

#### Identify outliers
1. To identify outliers we will use the IQR method
2. In this case we opted to only consider the extreme upper (Q3+ 3*IQR) and extreme lower bounds (Q1 -3*IQR)

Identify outliers in image width 

In [ ]:
# Identify outliers in image width
q3_width = metadata['image_width'].quantile(0.75)
q1_width = metadata['image_width'].quantile(0.25)
iqr_width = q3_width - q1_width
upper_bound_width = q3_width + 3 * iqr_width
lower_bound_width = q1_width - 3 * iqr_width
print(f"Upper bound for image width: {upper_bound_width}")
print(f"Lower bound for image width: {lower_bound_width}")

In [ ]:
# Identify number of images with width greater than upper bound --> around 12k total images
outlier_width_images = metadata[metadata['image_width'] > upper_bound_width]
print(f"Number of images with width greater than upper bound: {len(outlier_width_images)}")
print(f"% of images with width greater than upper bound: {len(outlier_width_images)/len(metadata) * 100:.2f}%")
# Display outlier images
outlier_width_images[['full_path', 'image_width']].head(10)


In [ ]:
# Distribution of global width outliers across phylum

# Obtain distribution of outlier images across phylum
outlier_counts_by_phylum = outlier_width_images['phylum'].value_counts().reset_index()
outlier_counts_by_phylum.columns = ['Phylum', 'Outlier Image Count']


# Add % of outlier images per phylum (relative to total outliers)
total_outliers = len(outlier_width_images)
outlier_counts_by_phylum['% of Outlier Images'] = (
    outlier_counts_by_phylum['Outlier Image Count'] / total_outliers * 100
).round(2)

outlier_counts_by_phylum.set_index('Phylum', inplace=True)

# Display the result
outlier_counts_by_phylum.sort_values(by='Outlier Image Count', ascending=False)


In [ ]:
# Distribution of global width outliers across family

outlier_counts_by_family = outlier_width_images['family'].value_counts().reset_index()
outlier_counts_by_family.columns = ['Family', 'Outlier Image Count']

# Add % of outlier images per family (relative to total outliers)
outlier_counts_by_family['% of Outlier Images'] = (
    outlier_counts_by_family['Outlier Image Count'] / total_outliers * 100
).round(2)

outlier_counts_by_family.set_index('Family', inplace=True)

# Display the result
outlier_counts_by_family.sort_values(by='Outlier Image Count', ascending=False)


In [ ]:
# Display the top two images with biggest width

top2_global_width_outliers = outlier_width_images.sort_values('image_width', ascending=False).head(2)[["full_path", "image_width"]]
for i in top2_global_width_outliers.index:
    img = Image.open(top2_global_width_outliers.loc[i, 'full_path'])
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Width: {top2_global_width_outliers.loc[i, 'image_width']}")
    plt.show()
    img.show()


In [ ]:
# Display two random images with average width
avg_width = metadata['image_width'].mean()
images_with_avg_width = metadata.loc[(metadata['image_width'] < avg_width + 10) & (metadata['image_width'] > avg_width - 10) ].sample(2)

for i in images_with_avg_width.index:
    img = Image.open(images_with_avg_width.loc[i, 'full_path'])
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Width: {images_with_avg_width.loc[i, 'image_width']}")
    plt.show()
    img.show()

Identify outliers in image height 

In [ ]:
# Identify outliers in image height
q3_height = metadata['image_height'].quantile(0.75)
q1_height = metadata['image_height'].quantile(0.25)
iqr_height = q3_height - q1_height
upper_bound_height = q3_height + 3 * iqr_height
lower_bound_height = q1_height - 3 * iqr_height
print(f"Upper bound for image width: {upper_bound_height}")
print(f"Lower bound for image width: {lower_bound_height}")

In [ ]:
# Identify number of images with height greater than upper bound --> around 12k total images
outlier_height_images = metadata[metadata['image_height'] > upper_bound_height]
print(f"Number of images with height greater than upper bound: {len(outlier_height_images)}")
print(f"% of images with height greater than upper bound: {len(outlier_height_images)/len(metadata) * 100:.2f}%")
# Display outlier images
outlier_height_images[['full_path', 'image_height']].head(10)

In [ ]:
# Distribution of global height outliers across phylum
# Obtain distribution of outlier images across phylum
outlier_counts_by_phylum_height = outlier_height_images['phylum'].value_counts().reset_index()
outlier_counts_by_phylum_height.columns = ['Phylum', 'Outlier Image Count']
# Add % of outlier images per phylum (relative to total outliers)
total_outliers_height = len(outlier_height_images)
outlier_counts_by_phylum_height['% of Outlier Images'] = (
    outlier_counts_by_phylum_height['Outlier Image Count'] / total_outliers_height * 100
).round(2)
outlier_counts_by_phylum_height.set_index('Phylum', inplace=True)

print(f"Total number of phylum with outliers: {len(outlier_counts_by_phylum_height)}")
# Display the result
outlier_counts_by_phylum_height.sort_values(by='Outlier Image Count', ascending=False)

In [ ]:
# Distribution of global height outliers across family
outlier_counts_by_family_height = outlier_height_images['family'].value_counts().reset_index()
outlier_counts_by_family_height.columns = ['Family', 'Outlier Image Count']
# Add % of outlier images per family (relative to total outliers)
outlier_counts_by_family_height['% of Outlier Images'] = (
    outlier_counts_by_family_height['Outlier Image Count'] / total_outliers_height * 100
).round(2)
outlier_counts_by_family_height.set_index('Family', inplace=True)

print (f"Total number of families with outliers: {len(outlier_counts_by_family_height)}")
# Display the result
outlier_counts_by_family_height.sort_values(by='Outlier Image Count', ascending=False)

In [ ]:
# Display the top two images with biggest height
top2_global_height_outliers = outlier_height_images.sort_values('image_height', ascending=False).head(2)[["full_path", "image_height"]]
for i in top2_global_height_outliers.index:
    img = Image.open(top2_global_height_outliers.loc[i, 'full_path'])
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Height: {top2_global_height_outliers.loc[i, 'image_height']}")
    plt.show()
    img.show()

In [ ]:
# Display two random images with average height
avg_height = metadata['image_height'].mean()
images_with_avg_height = metadata.loc[(metadata['image_height'] < avg_height + 10) & (metadata['image_height'] > avg_height - 10) ].sample(2)
for i in images_with_avg_height.index:
    img = Image.open(images_with_avg_height.loc[i, 'full_path'])
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Height: {images_with_avg_height.loc[i, 'image_height']}")
    plt.show()
    img.show()

Identify combined outliers in both image height and width

In [ ]:
# Identify number of images that are outliers in both dimensions - width and height
width_height_outliers = metadata[
    (metadata['image_width'] > upper_bound_width) &
    (metadata['image_height'] > upper_bound_height)
]

print(f"Images that are outliers in both width and height: {len(width_height_outliers)}")

In [ ]:
# Display the top 10 images with biggest width and height
width_height_outliers[['full_path', 'image_width', 'image_height']].sort_values(['image_width', 'image_height'], ascending=False).head(10)

Identify number of images evaluated as outliers in terms of values of pixels

In [ ]:
#Identify number of images evaluated as outliers in terms of values of pixels
metadata['pixel_count'] = metadata['image_width'] * metadata['image_height']
# Identify outliers in pixel count
q3_pixel = metadata['pixel_count'].quantile(0.75)
q1_pixel = metadata['pixel_count'].quantile(0.25)
iqr_pixel = q3_pixel - q1_pixel
upper_bound_pixel = q3_pixel + 3 * iqr_pixel
lower_bound_pixel = q1_pixel - 3 * iqr_pixel
print(f"Upper bound for pixel count: {upper_bound_pixel}")
print(f"Lower bound for pixel count: {lower_bound_pixel}")

In [ ]:
# Identify number of images with pixel count greater than upper bound --> around 12k total images
outlier_pixel_images = metadata[metadata['pixel_count'] > upper_bound_pixel]
print(f"Number of images with pixel count greater than upper bound: {len(outlier_pixel_images)}")
print(f"% of images with pixel count greater than upper bound: {len(outlier_pixel_images)/len(metadata) * 100:.2f}%")

> Despite using an extreme upper bound for pixel count, we still have too many images being evaluated as outliers 

> Perhaps using a different method to evaluate the number of outliers would be helpful 

In [ ]:
#Plot histogram of pixel count
plt.figure(figsize=(10, 6))
sns.boxplot(metadata['pixel_count'])
plt.xlabel('Pixel Count')
plt.title('Boxplot of Pixel Counts')
plt.show()


### Non-RGB Images

#### Identify the Non-RGB Images

In [ ]:
# Function to check image mode (e.g., 'RGB', 'L', etc.)
def check_mode(img_path):
    try:
        with Image.open(img_path) as img:
            return img.mode
    except:
        return None

# Apply function to full_path column
metadata['image_mode'] = metadata['full_path'].apply(check_mode)

# Optional: count the different modes
print(metadata['image_mode'].value_counts())


Visualize L mode images side by side

In [ ]:
# source https://stackoverflow.com/questions/41793931/plotting-images-side-by-side-using-matplotlib
# source https://keestalkstech.com/plotting-a-grid-of-pil-images-in-jupyter/


def display_images(image_paths, titles=None):
    """Display images in a grid."""
    n = len(image_paths)
    cols = 3
    rows = (n + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    axes = axes.flatten()
    
    for i, img_path in enumerate(image_paths):
        img = Image.open(img_path)
        axes[i].imshow(img)
        axes[i].axis('off')
        if titles:
            axes[i].set_title(titles[i])
    
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()
    
# Filter images with mode 'L'
l_images = metadata[metadata['image_mode'] == 'L']
# Display a sample of L mode images
sample_l_images = l_images.sample(6)
image_paths = sample_l_images['full_path'].tolist()
titles = sample_l_images['file_path'].tolist()
display_images(image_paths, titles)


Visualize CMYK mode images side by side

In [ ]:
# Filter images with mode 'CMYK'
cmyk_images = metadata[metadata['image_mode'] == 'CMYK']
# Display a sample of L mode images
sample_cmyk_images = cmyk_images.sample(6)
cmyk_image_paths = sample_cmyk_images['full_path'].tolist()
titles = sample_cmyk_images['file_path'].tolist()
display_images(cmyk_image_paths, titles)

#### Convert non RGB images to RGB images

> We can do it when using the image_dataset_from_directory function with color_mode="rgb"

### Identified Issues and Required Actions for Preprocessing (Alex + Joao)

| **Issue**                         | **Description**                                                                 | **Action Required**                                                                                                              | **When to implement it**                                      |
|----------------------------------|----------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------|
| **`1. Class Imbalance`**`             | Some families and phyla dominate in image count                                 | Use class weighting, data augmentation, or sampling strategies                                                                  | **After split** – Needs to be applied only on the training set |
| **2. Non-RGB Images**               | Some images are grayscale or have modes like "L", "P", etc.                     | Convert all images to RGB to standardize model input                                                                            | **After split** – During the loading of training, val and test datasets using the `image_dataset_from_directory` function with color_mode="rgb"       |
| **3. Corrupted or Unreadable Images** | Not an issue                                                                     | No action                                                                                                                       | –                                                             |
| **4. Very Large Images (Pixel Count)** | Some images have extremely high resolution → high memory cost                   | 1. Resize all images with aspect ratio preserved. 2. Apply padding or center-cropping.                       | **Inside the model** – Done by `Resizing()` (with crop_to_aspect_ratio=True or pad_to_aspect_ratio=True) and `CenterCrop()` layer  |
| **5. High Variability in Dimensions** | Lots of different resolutions and aspect ratios                                 | 1. Resize all images with aspect ratio preserved. 2. Apply padding or center-cropping.                                          | **Inside the model** – Done by `Resizing()` (with crop_to_aspect_ratio=True or pad_to_aspect_ratio=True) and `CenterCrop()` layer           |
| **6. Unusual Aspect Ratios**        | Some images are much taller or wider than typical                               | 1. Resize all images with aspect ratio preserved. 2. Apply padding or center-cropping.                                          | **Inside the model** – Done by `Resizing()` (with crop_to_aspect_ratio=True or pad_to_aspect_ratio=True) and `CenterCrop()` layer|
| **7. Brightness and Exposure Variation** | Some images are too dark or washed out                                         | Apply brightness/contrast normalization (optional)                                                                             | **During training only** – As part of data augmentation        |
| **`8. Noise label images`**      | Some images are not correctly representative of the phylum_family classes they are in | Consider removing or labeling separately if they harm generalization                                                           | **Before split** – Ensures training and evaluation consistency |
| **9. Metadata-Image Mismatches**    | (Not an issue) Some files in metadata don’t exist, or vice-versa               | No action                                                                                                                       | –                                                             |
| **10. Pixel Intensity Scaling**      | 1. Raw pixel values are in [0, 255] 2. Feeding those into a model can cause issues | Normalize pixel values to range [0, 1]                                                                                          | **Inside the model** – Done automatically by `Rescaling()`     |


## 2. Preprocessing stage

###  2.1 Content inconsistency

1. In this section, we will explain our strategy to deal with images that are clearly mismatched or not semantically aligned with their labeled phylum_family category
2. These can hurt model performance specially with pre-trained models
3. The model might learn to associate certain features with the wrong labels
4. To tackle this issue we ill use the Confidence Learning Methodology

#### Confidence Learning methodology
##### 1. Overview 
1. Confidence Learning (CL) is a data-centric framework for handling noisy labels in datasets. Its core ideas are:

2. Joint Distribution Estimation: CL estimates the joint distribution between the (noisy) observed labels and the (unknown) true labels using out‐of‐sample predicted probabilities. These probabilities are typically derived from a classifier (e.g., a CNN with a softmax output).

3. Class-Conditional Noise Assumption: It assumes that label noise is class-conditional—that is, the probability of a label being incorrect depends mainly on the class rather than the specific input features.

4. Thresholding & Ranking: By comparing predicted class probabilities (from out‐of‐sample predictions) with the given labels, CL uses per‑class thresholding to identify examples whose predictions are highly inconsistent with their assigned labels. The method then ranks these examples (by “self_confidence” or normalized margin) to flag likely label errors.

5. Model-Agnostic: Although the paper uses neural networks (and often a ResNet-type architecture) to generate probabilities, CL is designed to work with any classifier that outputs a probability distribution.

6. Key Requirement – Out‑of‑Sample Predictions:
- A critical component of CL is that it must use out-of-sample predicted probabilities. This means the predicted probabilities for each sample should be generated by a model that has not been trained on that particular sample. This approach avoids overfitting—where a model might be overly confident (and incorrectly so) on the training data—and instead provides unbiased estimates of uncertainty.

- Why Needed? In-sample predictions are often overconfident because the model may memorize the training data (including its noise), leading to unreliable probability estimates. By contrast, out-of-sample predictions (e.g., via cross-validation) ensure that each prediction better reflects the model’s uncertainty and can reveal true mismatches between the visual content and the assigned label.


(Reference: Northcutt et al., "Confident Learning: Estimating Uncertainty in Dataset Labels") -  https://arxiv.org/abs/1911.00068



##### 2. Clean Lab package

The Cleanlab package is built upon the principles of Confidence Learning. Specifically:

1. Implementation of CL: Cleanlab provides functions (such as find_label_issues()) that implement the CL algorithms. It calculates quantities like the "confident joint" based on the predicted probabilities and uses thresholding to determine which labels are likely erroneous.

2. Model-Agnostic Data Cleaning: Cleanlab works with any model that outputs class probabilities. Whether you use a lightweight CNN in Keras or any other classifier, Cleanlab uses the model’s predictions to flag label issues, enabling you to clean the dataset before training your final model.

3. Experimental Utility: The Cleanlab package is designed to be integrated into a typical machine learning workflow. For image datasets, it accepts predicted probabilities (from out-of‑sample predictions, ideally obtained via cross‑validation) and outputs a ranking of the samples with suspected label errors. This ranking can then guide manual inspection and removal of errors.

References: 
1) https://cleanlab.ai/blog/label-errors-image-datasets/ 
2) https://docs.cleanlab.ai/stable/cleanlab/models/keras.html#module-cleanlab.models.keras
3) https://github.com/cleanlab/cleanlab/tree/master/cleanlab

#### Approach to remove noisy label images

1. Identify images that are most likely to be label errors using the Clean Lab Package
2. Select top X for manual review and potential removal

#### Experimenting methodology with smaller subset of data 
##### Creating subset of data with only 5 phylum_family folders

In [ ]:
# Set the source and destination
source_dir = 'data'  # original folder with 202 class subfolders
dest_dir = 'data_subset'  # new folder for 10 random classes

# Create the destination directory if it doesn't exist
os.makedirs(dest_dir, exist_ok=True)

# Get all class subfolders
all_classes = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]

# Randomly select 10 classes
num_selected_classes = 5
selected_classes = random.sample(all_classes, num_selected_classes)
print("Selected classes:", selected_classes)

# Copy each selected class folder
for class_name in selected_classes:
    src_path = os.path.join(source_dir, class_name)
    dst_path = os.path.join(dest_dir, class_name)
    shutil.copytree(src_path, dst_path)

print("✅ 5 random classes copied to 'data_subset/'")


##### Step 1: Load all labeled data

In [ ]:
dataset = image_dataset_from_directory(
    'data_subset',                  
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    image_size=(224, 224),
    batch_size=32,
    interpolation='bilinear',
    shuffle=True
)

class_names = dataset.class_names
num_classes = len(class_names)

##### Step 2: Convert to NumPy arrays (X, y)

In [ ]:
# We need to convert the dataset to numpy arrays for cleanlab. It requires the data to be in numpy format.
# source: https://stackoverflow.com/questions/63513478/low-level-tensorflow-dataset-as-numpy-iterator-returns-dicts-instead-of-numpy
X_list, y_list = [], []

for images, labels in dataset.as_numpy_iterator(): # to avoid loading all images into memory at once. 
    X_list.append(images)
    y_list.append(labels)

X = np.concatenate(X_list)
y = np.concatenate(y_list)

##### Step 3. Define the model 

Why did we select the `MobileNetV2` architecture as our pre trained model?
1. Based on the model comparison table in the Keras documentation (https://keras.io/api/applications/), achieves a competitive top-1 accuracy while having fewer parameters and faster inference times
2. Hence MobileNetV2 provides sufficiently strong performance for our task while being computationally efficient.

In [ ]:
# Define the model
def build_model(input_shape=(224, 224, 3), num_classes=num_selected_classes):
    inputs = Input(shape=input_shape)
    x = Rescaling(1. / 255)(inputs)
    x = MobileNetV2(include_top=False, pooling='avg', weights='imagenet')(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    return Model(inputs, outputs)



##### Step 4. Train the model 

In [ ]:
# Convert one-hot encoded labels to integer labels
y_int = np.argmax(y, axis=1)

pred_probs = np.zeros((len(X), num_classes))  # To store OOS predicted probabilities

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

for train_idx, val_idx in skf.split(X, y_int):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train = y[train_idx]  # you can still use the one-hot version for training if needed

    clf = KerasWrapperModel(
    model=build_model,
    model_kwargs={'input_shape': (224, 224, 3), 'num_classes': num_classes},
    compile_kwargs={
        'optimizer': 'adam',
        'loss': 'categorical_crossentropy',  # Changed from sparse_categorical_crossentropy
        'metrics': ['accuracy']
    }
    )

    clf.fit(X_train, y_train, epochs=3, batch_size=32, verbose=1)

    preds = clf.predict_proba(X_val)
    pred_probs[val_idx] = preds



##### Step 5: Use Cleanlab to Identify Label Issues

In [ ]:
from cleanlab.filter import find_label_issues

# Find label issues using the predicted probabilities
ranked_label_issues = find_label_issues(
    labels=y_int,
    pred_probs=pred_probs,
    return_indices_ranked_by="self_confidence"
)

print(f"Found {len(ranked_label_issues)} potential label issues.")
print("Here are the indices of the top 15 most likely label errors:\n"
      f"{ranked_label_issues[:15]}")


In [ ]:
# Dsiplay the top n images with the most likely label errors

def display_ranked_label_errors(X, y_int, pred_probs, class_names, ranked_indices, n=5):
    """
    Display the top n images with the highest label error likelihood as determined by Cleanlab.

    Parameters:
      X               : numpy array of images, shape (n_samples, height, width, channels)
      y_int           : numpy array of true labels as integers (1D array, length n_samples)
      pred_probs      : numpy array of predicted probability distributions, shape (n_samples, num_classes)
      class_names     : list of class names corresponding to the integer labels
      ranked_indices  : array/list of indices sorted in descending order of label error likelihood
      n               : number of images to display (default is 5)
    """
    # Select the top n indices based on label error ranking
    top_indices = ranked_indices[:n]
    
    # Compute grid layout: at most 3 columns
    cols = min(n, 3)
    rows = (n + cols - 1) // cols

    fig, axes = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))
    
    # Ensure axes is a 1D array for easy iteration
    if rows * cols > 1:
        axes = axes.flatten()
    else:
        axes = [axes]

    # Loop through top indices and display each image
    for i, idx in enumerate(top_indices):
        # X[idx] is the image array; convert to uint8 for proper display if needed.
        axes[i].imshow(X[idx].astype('uint8'))
        axes[i].axis('off')
        # Determine predicted and true classes
        pred_label = class_names[np.argmax(pred_probs[idx])]
        true_label = class_names[y_int[idx]]
        axes[i].set_title(f"Pred: {pred_label}\nTrue: {true_label}\nIdx: {idx}")
    
    # Hide any unused axes (if grid has more cells than images)
    for j in range(i + 1, rows * cols):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()
# Display the top 15 most likely label errors
display_ranked_label_errors(X, y_int, pred_probs, class_names, ranked_label_issues, n=15)

#### Experimenting methodology on entire dataset


##### Step 1: Load all labeled data

In [ ]:
# Load the full dataset
full_dataset = image_dataset_from_directory(
    'data',                           
    labels='inferred',
    label_mode='categorical',         
    color_mode='rgb',
    image_size=(224, 224),
    batch_size=32,
    interpolation='bilinear',
    shuffle=True                      # For initial loading, you can shuffle or disable for consistency
)

class_names = full_dataset.class_names
num_classes = len(class_names)
print("Class names:", class_names)


##### Step 2: Convert to NumPy arrays (X, y)

In [ ]:
X_list_full, y_list_full = [], []

for images, labels in full_dataset.as_numpy_iterator(): # to avoid loading all images into memory at once. 
    X_list_full.append(images)
    y_list_full.append(labels)

X_full = np.concatenate(X_list_full)
y_full = np.concatenate(y_list_full)

##### Step 3. Define the model 

Why did we select the `MobileNetV2` architecture as our pre trained model?
1. Based on the model comparison table in the Keras documentation (https://keras.io/api/applications/), achieves a competitive top-1 accuracy while having fewer parameters and faster inference times
2. Hence MobileNetV2 provides sufficiently strong performance for our task while being computationally efficient.

Why are we using the Functional API method and not the class method to design our model?
1. We do it to create a model that is cloneable and compatible with Cleanlab’s wrappers.
2. Cleanlab’s wrappers is a scikit‑learn tool that relies on being able to clone the model, which is easier to do with a function that returns a functional or Sequential model

In [ ]:
# Define the model
def build_model(input_shape=(224, 224, 3), num_classes=num_classes):
    inputs = Input(shape=input_shape)
    # Reschaling layer
    x = Rescaling(1. / 255)(inputs)
    x = MobileNetV2(include_top=False, pooling='avg', weights='imagenet')(x) # produces a single flat vector from its feature maps
    # Dense layer
    outputs = Dense(num_classes, activation='softmax')(x)
    return Model(inputs, outputs)

##### Step 4. Train the model 

In [ ]:
# Convert one-hot encoded labels to integer labels
y_int_full = np.argmax(y_full, axis=1)

pred_probs = np.zeros((len(X_full), num_classes))  # To store OOS predicted probabilities

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

for train_idx, val_idx in skf.split(X_full, y_int_full):
    X_train, X_val = X_full[train_idx], X_full[val_idx]
    y_train = y_full[train_idx]  # you can still use the one-hot version for training if needed

    clf = KerasWrapperModel(
    model=build_model,
    model_kwargs={'input_shape': (224, 224, 3), 'num_classes': num_classes},
    compile_kwargs={
        'optimizer': 'adam',
        'loss': 'categorical_crossentropy',  # Changed from sparse_categorical_crossentropy
        'metrics': ['accuracy']
    }
    )

    clf.fit(X_train, y_train, epochs=7, batch_size=32, verbose=1)

    preds = clf.predict_proba(X_val)
    pred_probs[val_idx] = preds



In [ ]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report
)

# 1. Convert predicted probabilities to predicted labels via argmax
predicted_labels = pred_probs.argmax(axis=1)

# 2. Basic Accuracy
acc = accuracy_score(y_int_full, predicted_labels)
print(f"Accuracy: {acc:.3f}")

# 3. Precision, Recall, F1 (macro-averaged, but you can also use 'weighted' or 'micro')
prec = precision_score(y_int_full, predicted_labels, average='macro')
rec = recall_score(y_int_full, predicted_labels, average='macro')
f1 = f1_score(y_int_full, predicted_labels, average='macro')

print(f"Precision (macro): {prec:.3f}")
print(f"Recall (macro):    {rec:.3f}")
print(f"F1 Score (macro):  {f1:.3f}")

# 4. Classification Report (shows per-class performance)
print("Classification Report:")
print(classification_report(y_int_full, predicted_labels, target_names=class_names))


> As we can see the model is not performing well with an overall accuracy of around 0.039 and an F1 Score (macro of 0.008)

> Even though the results are quite poor, the helper model is just being used to identify potential label errors with the out of sample predicted probabilities

> Considering the goal of this stage and that we are still not in the modeling stage, we can still use the predicted probabilities to find label errors

##### Step 5: Use Cleanlab to Identify Label Issues

In [ ]:
from cleanlab.filter import find_label_issues

# Find label issues using the predicted probabilities
ranked_label_issues = find_label_issues(
    labels=y_int_full,
    pred_probs=pred_probs,
    return_indices_ranked_by="self_confidence"
)

print(f"Found {len(ranked_label_issues)} potential label issues.")
print("Here are the indices of the top 15 most likely label errors:\n"
      f"{ranked_label_issues[:15]}")


In [ ]:
# Visualize the top n images with the most likely label errors
# Display the top 15 most likely label errors
display_ranked_label_errors(X_full, y_int_full, pred_probs, class_names, ranked_label_issues, n=200)

Understanding the obtained results

For each image \( x_k \) with a provided (noisy) label \( y_k \), your model (via cross-validation) produces a probability vector:

$$
\hat{p}(y \mid x_k) = \big[\hat{p}(y=0 \mid x_k),\ \hat{p}(y=1 \mid x_k),\ \dots,\ \hat{p}(y=201 \mid x_k)\big].
$$

**Self-confidence** for that image is defined as the **predicted probability assigned to its provided label**.  
For instance, if the provided label (the "true" label from your dataset) for \( x_k \) is class \( i \), the self-confidence is:

$$
\text{Self-confidence} = \hat{p}(y = i \mid x_k).
$$

- A **high self-confidence** (e.g., 0.20) means the model is very confident that the image belongs to the provided label.
- A **low self-confidence** (e.g., 0.02) suggests that the model doubts the label—this image is likely to be mislabeled or semantically mismatched.







Function to determine the top n label error images for each class

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

def display_label_errors_for_class_range(
    X, 
    y_int, 
    pred_probs, 
    class_names, 
    ranked_indices, 
    start, 
    end, 
    n=10
):
    """
    Display the top n label error images for each class in a specified range,
    including the self-confidence (predicted probability) for the given label.
    
    Parameters:
      X              : numpy array of images, shape (n_samples, height, width, channels).
      y_int          : 1D numpy array of true labels as integers (length = n_samples).
      pred_probs     : numpy array of predicted probability distributions, shape (n_samples, num_classes).
      class_names    : list of class names corresponding to the label indices.
      ranked_indices : array or list of indices sorted by label error likelihood (most suspicious first).
      start          : starting index (inclusive) in class_names (e.g., 0).
      end            : ending index (exclusive) in class_names (e.g., 3).
      n              : number of images to display for each class (default is 10).
    """
    for class_idx in range(start, end):
        target_class = class_names[class_idx]
        print(f"\nDisplaying top {n} label error images for class '{target_class}' (Index {class_idx})")
        
        # Filter the ranked indices to only include samples with true label equal to target_class.
        filtered_indices = [idx for idx in ranked_indices if y_int[idx] == class_idx]
        
        if not filtered_indices:
            print(f"No label issues detected for class '{target_class}'.")
            continue
        
        # Select the top n indices from this filtered list.
        top_indices = filtered_indices[:n]
        
        # Set up a grid with at most 3 columns.
        cols = min(n, 3)
        rows = (n + cols - 1) // cols
        fig, axes = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))
        
        # Flatten the axes for easier iteration.
        if rows * cols > 1:
            axes = axes.flatten()
        else:
            axes = [axes]
            
        # Display each image with its details.
        for i, idx in enumerate(top_indices):
            axes[i].imshow(X[idx].astype('uint8'))
            axes[i].axis('off')
            pred_label = class_names[np.argmax(pred_probs[idx])]
            true_label = class_names[y_int[idx]]
            # Self-confidence for the true label.
            confidence = pred_probs[idx][y_int[idx]]
            axes[i].set_title(f"Pred: {pred_label}\nTrue: {true_label}\nConf: {confidence:.5f}\nIdx: {idx}")
        
        # Turn off any extra axes.
        for j in range(i + 1, rows * cols):
            axes[j].axis('off')
        
        plt.tight_layout()
        plt.show()


In [ ]:
# Display the top 10 label error images for each class in the range of 0 to 3
display_label_errors_for_class_range(
    X_full, 
    y_int_full, 
    pred_probs, 
    class_names, 
    ranked_label_issues, 
    start=22, 
    end=26, 
    n=10
)

In [ ]:
print("Predicted probability vector:\n", pred_probs[4254])

Analyzing Confidence level of n first images of each phylum_family class for correct predictions

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

def display_correct_predictions_by_class_range(
    X, 
    y_int, 
    pred_probs, 
    class_names, 
    start, 
    end, 
    n=20
):
    """
    Display the first n images for each class in the specified range that are correctly predicted 
    by the model. For each image, the title displays:
      - The predicted label,
      - The true label,
      - The confidence level (predicted probability for the given true label),
      - And the index of the image.
    
    Parameters:
      X           : numpy array of images, shape (n_samples, height, width, channels).
      y_int       : 1D numpy array of true labels as integers (length = n_samples).
      pred_probs  : numpy array of predicted probability distributions, shape (n_samples, num_classes).
      class_names : list of class names corresponding to the label indices.
      start       : starting index (inclusive) in class_names.
      end         : ending index (exclusive) in class_names.
      n           : number of images to display for each class (default is 20).
    """
    for class_idx in range(start, end):
        target_class = class_names[class_idx]
        print(f"\nDisplaying first {n} correct predictions for class '{target_class}' (Index {class_idx})")
        
        # Find indices where the true label equals the target class.
        indices = np.where(y_int == class_idx)[0]
        
        # Filter to keep only those indices where the predicted label matches the true label.
        correct_indices = [idx for idx in indices if np.argmax(pred_probs[idx]) == y_int[idx]]
        
        if len(correct_indices) == 0:
            print(f"No correct predictions found for class '{target_class}'.")
            continue
        
        # Select the first n correct predictions for this class.
        top_indices = correct_indices[:n]
        
        # Define grid layout (here, up to 5 columns)
        cols = min(len(top_indices), 5)
        rows = (len(top_indices) + cols - 1) // cols
        fig, axes = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))
        
        # Flatten axes for ease of iteration.
        if rows * cols > 1:
            axes = axes.flatten()
        else:
            axes = [axes]
        
        for i, idx in enumerate(top_indices):
            axes[i].imshow(X[idx].astype('uint8'))
            axes[i].axis('off')
            pred_label = class_names[np.argmax(pred_probs[idx])]
            true_label = class_names[y_int[idx]]
            confidence = pred_probs[idx][y_int[idx]]
            axes[i].set_title(f"Pred: {pred_label}\nTrue: {true_label}\nConf: {confidence:.4f}\nIdx: {idx}")
            
        # Hide any unused subplots.
        for j in range(i + 1, rows * cols):
            axes[j].axis('off')
        
        plt.suptitle(f"Class: {target_class}", fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.show()


In [ ]:
display_correct_predictions_by_class_range(
    X_full, 
    y_int_full, 
    pred_probs, 
    class_names, 
    start=0, 
    end=3, 
    n=20
)


##### After using Cleanlab to detect potential label errors: 
1. Use find_label_issues() with return_indices_ranked_by="self_confidence"
2. Display the top 100–500 images flagged as errors
3. Manually tag each one as: keep / remove / relabel
4. Save a clean version of X_clean, y_clean, and optionally log what was removed
5. Only then proceed to train your final model

### 2.2 Spliting data into train, val and test datasets


In [ ]:
# Using stratufy to maintain the distribution of classes in the train, validation, and test sets
train_df, val_test_df = train_test_split(metadata, test_size=0.3, stratify=metadata['family'], random_state=42)
val_df, test_df = train_test_split(val_test_df, test_size=0.5, stratify=val_test_df['family'], random_state=42)

In [ ]:
# Create folder structure and move/copy images
## source https://www.kaggle.com/discussions/general/111208
original_base_dir = 'data/'

# Helper function to move files into train/val/test folder structure
def move_images(df, split_name):
    for _, row in df.iterrows():
        class_name = row['family']
        src_path = os.path.join(original_base_dir, row['file_path'])
        print(src_path)
        dst_dir = os.path.join('data_split', split_name, class_name)
        dst_path = os.path.join(dst_dir, os.path.basename(src_path))
        
        os.makedirs(dst_dir, exist_ok=True)  # Create class folder if it doesn't exist
        
        if os.path.exists(src_path):
            shutil.copy(src_path, dst_path)  # Or use shutil.move() if you want to move instead of copy
        else:
            print(f"Missing file: {src_path}")

In [ ]:
# Create directories
os.makedirs('data_split/train', exist_ok=True)
os.makedirs('data_split/validation', exist_ok=True)
os.makedirs('data_split/test', exist_ok=True)


# Move images to their corresponding folders
move_images(train_df, 'train')
move_images(val_df, 'validation')
move_images(test_df, 'test')


### 2.3 Class imbalance strategy 

#### Transformation Oversampling 
1. Definition: The image transformations used for data augmentation can also be applied as part of an oversampling method that can
be employed to balance an imbalanced training dataset as proposed by Wang
and Perez [15].
2. Steps
    - Identify the minority classes in the dataset (those with fewer images).
    - For each image (patch Pj) in the minority class, apply a set of transformations:
        - 90-degree rotation
        - Gaussian blur (σ = 1)
        - Flipping (horizontal/vertical)
    - Ensure transformations aren’t repeatedly applied to the same image unless all other transformations have already been used.
        - Apply each transformation once per image, not apply same transformation 3 times to the same image. 

    - Add the resulting transformed images to the training dataset to balance the class distribution.


##### How to determine the minority classes and size they should be increased to

Rule 1:

$$
r_{100} : s(l) = s(l_{\text{maj}})
$$

Rule 2: 
$$
r_{50} : s(l) =
\begin{cases}
\frac{s(l_{\text{maj}})}{2} & \text{if } s(l) < \frac{s(l_{\text{maj}})}{2} \\
s(l) & \text{else}
\end{cases}
$$

Rule 3:

$$
r_{50,75,100} : s(l) =
\begin{cases}
\frac{1}{2}s(l_{\text{maj}}) & \text{if } s(l) \leq \frac{1}{4}s(l_{\text{maj}}) \\
\frac{3}{4}s(l_{\text{maj}}) & \text{if } \frac{1}{4}s(l_{\text{maj}}) < M_k \leq \frac{1}{2}s(l_{\text{maj}}) \\
s(l_{\text{maj}}) & \text{else}
\end{cases}
$$

Check the distribution of images in the new folders

Training data

In [ ]:
# Number of images per phylum_family in train, validation, and test sets
# Show family_phylum as index and then image count as column
train_family_df = build_summary_df(train_df['family'].value_counts(), 'Family')
train_family_df['Phylum'] = train_family_df['Family'].map(family_phylum_map)
train_family_df['Image Count'] = train_family_df['Family'].map(train_df['family'].value_counts())
train_family_df['% of Total Images'] = (
    train_family_df['Image Count'] / len(train_df) * 100
).round(2)
train_family_df

In [ ]:
# Number of families per range of image count
# Create bins for image count
bins = [0, 21, 42, 63, 84, 105, 126, 147, np.inf]
labels = ['0-21', '22-42', '43-63', '64-84', '85-105', '106-126', '127-147', '148+']
train_family_df['Image Count Range'] = pd.cut(train_family_df['Image Count'], bins=bins, labels=labels)
# Count number of families in each range
image_count_range_counts = train_family_df['Image Count Range'].value_counts().reset_index()
image_count_range_counts.columns = ['Image Count Range', 'Number of Families/Classes']
# Sort by image count range
image_count_range_counts = image_count_range_counts.sort_values(by='Image Count Range')
# Display the result
image_count_range_counts.set_index('Image Count Range', inplace=True)
image_count_range_counts

> Considering the information above, 118/202 families have 0-21 images, which is 58.41% of the dataset.

> The biggest family has 210 images

> Considering that, if we apply the r_100 or the r_50, the upsample 118 families from 21 images to 210 images

### 2.4 Load and convert images for all 3 splits	